In [1]:
# Load libraries

In [2]:
import pandas as pd
import ipywidgets
from ipywidgets import interact
from ipywidgets import interact_manual

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
df=pd.read_csv("sample_data/books.csv")
df.head(5)

In [ ]:
df.info()

In [ ]:
df['language_code'].nunique()

In [ ]:
@interact
def book_by_author(x = list(df['authors'].value_counts().index)):
  df_s = df[df['authors']==x][['title', 'average_rating']].sort_values('average_rating', ascending=False)
  return df_s.head(5)

In [ ]:
# converting average rating column into categorical column
def num_into_obj(x):
    if x>=0 and x<=1:
        return 'between 0 and 1'
    elif x>1 and x<=2:
        return 'between 1 and 2'
    elif x>2 and x<=3:
        return 'between 2 and 3'
    elif x>3 and x<=4:
        return 'between 3 and 4'
    else:
        return 'between 4 and 5'
df['rating_obj'] = df['average_rating'].apply(num_into_obj)

In [ ]:
rating_df = pd.get_dummies(df['rating_obj'])
rating_df.head()

In [ ]:
language_df = pd.get_dummies(df['language_code'])
language_df.head()

In [ ]:
# concat both the data frames and set the title column as the index 
features = pd.concat([rating_df, language_df, df['average_rating'], df['ratings_count'], df['title']], axis=1)
features.set_index('title', inplace=True)
features.head()

In [ ]:
# for scaling the values of the data frame
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
features_scaled = min_max_scaler.fit_transform(features)

In [ ]:
# importing neighbours
from sklearn import neighbors

In [ ]:
# training the model
model = neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree', metric='euclidean')
model.fit(features_scaled)
dist, idlist = model.kneighbors(features_scaled)

In [ ]:
dist[1]

In [ ]:
idlist[1]

In [ ]:
@interact
def BookRecommender(book_name = list(df['title'].value_counts().index)):
    book_list_name = []
    book_id = df[df['title'] == book_name].index
    book_id = book_id[0]
    for newid in idlist[book_id]:
        book_list_name.append(df.loc[newid].title)
    return book_list_name

In [ ]:
df[df['title']=='King Lear']